<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/temporal-analysis/full_time/analyse_NY_mobility_poverty_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 2.07 ms (started: 2021-04-22 01:43:25 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=5937d23cf61f4e117eda1bef6a690e91864c3de9f5ae7fe1f5b95af816dafcc0
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 106 ms (started: 2021-04-22 01:44:18 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 291 ms (started: 2021-04-22 01:44:18 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 5.02 s (started: 2021-04-22 01:44:19 +00:00)


In [6]:
#sc.stop()

time: 1.36 ms (started: 2021-04-22 01:44:24 +00:00)


In [7]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 26.2 s (started: 2021-04-22 01:44:24 +00:00)


In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 1.68 ms (started: 2021-04-22 01:44:50 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, full_time_work_behavior_devices FROM clean_ny WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+-------------------------------+
|         cbg|                date|device_count|full_time_work_behavior_devices|
+------------+--------------------+------------+-------------------------------+
|360470064002|2020-02-03T00:00:...|          64|                              5|
|360810384001|2020-02-03T00:00:...|          84|                              8|
|360850170103|2020-02-03T00:00:...|         250|                             26|
|360050213021|2020-02-03T00:00:...|          92|                              6|
|360050253004|2020-02-03T00:00:...|          52|                              4|
|360470385003|2020-02-03T00:00:...|          37|                              5|
|360470406002|2020-02-03T00:00:...|          46|                              4|
|360470428003|2020-02-03T00:00:...|          84|                             13|
|360471130002|2020-02-03T00:00:...|          84|                              9|
|360811029001|2020-02-03T00:

In [11]:
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 60.2 ms (started: 2021-04-22 01:45:17 +00:00)


In [14]:
df_soc.show()

+------------+----------+------------+-------------------------------+
|         cbg|      date|device_count|full_time_work_behavior_devices|
+------------+----------+------------+-------------------------------+
|360470064002|2020-02-03|          64|                              5|
|360810384001|2020-02-03|          84|                              8|
|360850170103|2020-02-03|         250|                             26|
|360050213021|2020-02-03|          92|                              6|
|360050253004|2020-02-03|          52|                              4|
|360470385003|2020-02-03|          37|                              5|
|360470406002|2020-02-03|          46|                              4|
|360470428003|2020-02-03|          84|                             13|
|360471130002|2020-02-03|          84|                              9|
|360811029001|2020-02-03|         256|                             18|
|360050177022|2020-02-03|          53|                              1|
|36005

In [15]:
# Add completely home percentage column
#df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
# df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

#df_soc.show()

time: 60.4 ms (started: 2021-04-22 01:46:44 +00:00)


In [16]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 19.1 ms (started: 2021-04-22 01:46:44 +00:00)




---


**Demographic data**





---



In [17]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050175002|     1110| 55.22522522522523|14.774774774774773|
|360050141001|     1445| 36.74740484429066| 19.16955017301038|
|360050145001|     2003| 39.47667804323095|  8.53719420868697|
|360050075002|     2731|22.812156719150494|4.5404613694617355|
|360050418001|      936|18.859649122807017| 9.081196581196581|
|360050125001|     2935| 43.24693042291951|12.742759795570699|
|360050127012|     1061| 43.73232799245994|18.378887841658813|
|360050054003|     1031| 57.12900096993211| 28.12803103782735|
|360050027011|     1364|  65.9090909090909|14.589442815249267|
|360050458001|     1168| 37.15753424657534| 2.910958904109589|
|360050224011|     1291|19.907048799380327|27.420604182804027|
|360050389001|     1143| 53.94265232974911|30.708661417322837|
|360050071002|     1661|           37.4375|11.499096929

In [18]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+-------------------------------+-------------------------+------------------+
|         cbg|      date|device_count|full_time_work_behavior_devices|full_time_work_percentage|poverty_percentage|
+------------+----------+------------+-------------------------------+-------------------------+------------------+
|360470064002|2020-02-03|          64|                              5|                   7.8125| 19.22705314009662|
|360810384001|2020-02-03|          84|                              8|        9.523809523809524|12.291350531107739|
|360850170103|2020-02-03|         250|                             26|                     10.4|13.652802893309223|
|360050213021|2020-02-03|          92|                              6|        6.521739130434782| 43.07328605200946|
|360050253004|2020-02-03|          52|                              4|       7.6923076923076925|24.824120603015075|
|360470385003|2020-02-03|          37|                              5|  

In [39]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("full_time_work_percentage").withColumnRenamed('avg(full_time_work_percentage)','full_time_work_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("full_time_work_percentage").withColumnRenamed('avg(full_time_work_percentage)','full_time_work_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.full_time_work_percentage as top20_perc,
l.full_time_work_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 270 ms (started: 2021-04-22 01:51:43 +00:00)


**Load 2019 stats**

In [40]:
filename = '{}_social_summary_2019.csv'.format(city)
stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/filename))
stat_2019.show()
mean_2019 = stat_2019.collect()[1][7]


+-----+------------------+------------------+-----------------+------------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+-----------------+
|  _c0|             total|   completely_home|   part_time_work|    full_time_work|percentage_completely_home|percentage_part_time_work|percentage_full_time_work|norm_completely_home| norm_part_time_work| norm_full_time_work|            month|
+-----+------------------+------------------+-----------------+------------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+-----------------+
|count|         2290248.0|         2290248.0|        2290248.0|         2290248.0|                 2290248.0|                2290248.0|                2290248.0|           2290248.0|           2290248.0|           2290248.0|        2290248.0|
| mean| 80.34526610218632|28

In [41]:
# calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 23 ms (started: 2021-04-22 01:51:44 +00:00)


In [42]:
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 18.2 ms (started: 2021-04-22 01:51:44 +00:00)



**Visualizations** 



In [43]:
# create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': ['#FF4500','#a0aab4','#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')
# 

ThemeRegistry.enable('my_theme')

time: 13.5 ms (started: 2021-04-22 01:51:44 +00:00)


In [54]:
def create_chart(df) :
  chart = alt.Chart(df).transform_fold(
    fold=['Bottom 20 % Deviation', 'Top 20 % Deviation'],
    as_ = ['Poverty %','full_time_work']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = 'Date'),
      y= alt.Y('full_time_work:Q', title = 'Full Time %'),
      color='Poverty %:N'
    ).properties(width = 1000).interactive()

  rule = alt.Chart(pd.DataFrame({'Mean 2019':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='Mean 2019',
    size=alt.value(2),
    )
  
  text = alt.Chart({'values':[{ 'y': 1}]}).mark_text(
    text='Mean 2019', angle=0
    ).encode(
      y=alt.Y('y:Q'),opacity=alt.value(0.4)
    )

  return  chart + rule + text


time: 15.2 ms (started: 2021-04-22 01:54:52 +00:00)


In [45]:
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 4.76 ms (started: 2021-04-22 01:51:44 +00:00)


In [46]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,10.014082,4.626570,4.407534,-0.979979
1,2020-04-13,3.924350,4.518371,-1.682198,-1.088177
2,2020-06-24,5.279721,10.457516,-0.326828,4.850968
3,2020-06-08,4.370070,3.149351,-1.236479,-2.457198
4,2020-09-12,4.606030,7.439271,-1.000519,1.832723
...,...,...,...,...,...
361,2020-04-05,3.513350,3.306452,-2.093198,-2.300097
362,2020-05-01,3.659375,2.907268,-1.947174,-2.699280
363,2020-10-25,3.706099,7.528736,-1.900449,1.922187
364,2020-12-28,5.966273,4.846154,0.359724,-0.760395


time: 1min 18s (started: 2021-04-22 01:51:44 +00:00)


In [47]:
df = df.rename(columns={'top20_perc': 'Top 20 %', 'bottom20_perc': 'Bottom 20 %','top20_deviation': 'Top 20 % Deviation', 'bottom20_deviation': 'Bottom 20 % Deviation'})

time: 6.15 ms (started: 2021-04-22 01:53:02 +00:00)


In [48]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['Bottom 20 %', 'Top 20 %'],
    as_ = ['Poverty %','full_time_percentage']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('full_time_percentage:Q', title = 'Full Time %'), color='Poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 110 ms (started: 2021-04-22 01:53:02 +00:00)


In [49]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
#df['date'] = [datetime.datetime.date(d) for d in df['date']] 

time: 3.33 ms (started: 2021-04-22 01:53:03 +00:00)


Filtering based on important dates

**March 8, 2020:**
NYC issues guidelines to avoid densely packed buses, subways, or trains.

**March 12, 2020:**
Broadway shuts down.

**March 14, 2020:**
First two COVID-19 deaths in NYS.

**March 22, 2020:**
NYS on Pause Program begins, all non-essential workers must stay home.

**March 28, 2020:**
Governor Cuomo halts all nonessential construction sites in NYS.











In [55]:
start_date = '2020-3-5'
end_date = '2020-3-31'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1)
chart

alt.LayerChart(...)

time: 56.8 ms (started: 2021-04-22 01:54:58 +00:00)




**April 10, 2020:**
NYS records more COVID19 cases than any other country 









In [56]:
start_date = '2020-4-2'
end_date = '2020-4-24'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2)
chart
#df2

alt.LayerChart(...)

time: 54.6 ms (started: 2021-04-22 01:55:08 +00:00)



**July 6, 2020:** 
NYC begins Phase 3 of reopening, without indoor dining





In [57]:
start_date = '2020-7-1'
end_date = '2020-7-16'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3)
chart

alt.LayerChart(...)

time: 59.3 ms (started: 2021-04-22 01:55:56 +00:00)


**October 28, 2020:** New York tops a half-million confirmed COVID-19 cases


In [58]:
start_date = '2020-10-20'
end_date = '2020-11-8'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4)
chart

alt.LayerChart(...)

time: 58.4 ms (started: 2021-04-22 01:56:04 +00:00)
